# Example block-wise adaptation using FOTDA and BOTDA

In [1]:
import numpy as np
from numpy import unravel_index
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import ot
import scipy.io
import mne          
from mne.decoding import CSP
mne.set_log_level(verbose='warning') #to avoid info at terminal
import matplotlib.pyplot as pl
from random import seed
seed(30)
from MIOTDAfunctions import*

# get the functions from RPA package
import rpa.transfer_learning as TL
# pyriemann import
from pyriemann.classification import MDM
from pyriemann.estimation import Covariances
from pyriemann.utils.base import invsqrtm
import timeit

#ignore warning 
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)

In [2]:
results_acc=[]
results_all=[]
results_all_inv=[]

rango_cl = [0.1, 0.5, 1, 2, 5, 10, 20]
rango_e = [0.1, 0.5, 1, 2, 5, 10, 20]
metrica = 'sqeuclidean'
outerkfold = 10 # for faster online computation select a lower value
innerkfold = dict(nfold=10, train_size=0.8)

## Load and filter data 

In [3]:
fName = 'Data/DataSession1_S9.mat'
s = scipy.io.loadmat(fName)

Data_S1=s["X"]
Labels_S1=s["y"]
Labels_S1=np.squeeze(Labels_S1)

#filterting with mne
[nt, nc, ns]=np.shape(Data_S1)
Data_S1=np.reshape(Data_S1, [nt, nc*ns])
Data_S1=mne.filter.filter_data(Data_S1, 128, 8, 30)
Data_S1=np.reshape(Data_S1, [nt,nc,ns])

fName = 'Data/DataSession2_S9.mat'
s2 = scipy.io.loadmat(fName)

Data_S2=s2["X"]
Labels_S2=s2["y"]
Labels_S2=np.squeeze(Labels_S2)

#filterting with mne
[nt, nc, ns]=np.shape(Data_S2)
Data_S2=np.reshape(Data_S2, [nt, nc*ns])
Data_S2=mne.filter.filter_data(Data_S2, 128, 8, 30)
Data_S2=np.reshape(Data_S2, [nt,nc,ns])

### Learn CSP+LDA from source data (Data_S1)

In [4]:
Xtr = Data_S1
Ytr = Labels_S1
csp = CSP(n_components=6, reg='empirical', log=True, norm_trace=False, cov_est='epoch')
#learn csp filters
Gtr = csp.fit_transform(Xtr, Ytr)
#learn lda
lda = LinearDiscriminantAnalysis()
lda.fit(Gtr,Ytr)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

## for each run of 20 trials each learn and apply the transport

In [ ]:
for re in range(0,7):
    print('Running testing RUN={:1.0f}'.format(re))
    #testing run
    Xte = Data_S2[0+20*(re+1):20*(re+1)+20]
    Yte = Labels_S2[0+20*(re+1):20*(re+1)+20]
    #transportation set-prior data
    Xval = Data_S2[0:20*re+20]
    Yval = Labels_S2[0:20*re+20]
    
    #feature computation
    Gval = csp.transform(Xval)
    Gte = csp.transform(Xte)
    
    #evaluate SC  
    acc_sc = lda.score(Gte, Yte)
    
    # time
    start = timeit.default_timer()
    #evaluate SR
    Xtr2add = Data_S2[0:20*re+20]
    Ytr2add=Labels_S2[0:20*re+20]
    Xtr2 = np.vstack(((Xtr, Xtr2add)))
    Ytr2 = np.hstack(((Ytr, Ytr2add)))
        
    Ytr2 = Ytr2[len(Ytr2add):]
    Xtr2 = Xtr2[len(Ytr2add):]

    csp2 = CSP(n_components=6, reg='empirical', log=True, norm_trace=False, cov_est='epoch')
    #learn new csp filters
    Gtr2 = csp2.fit_transform(Xtr2,Ytr2)
    #learn new lda
    lda2 = LinearDiscriminantAnalysis()
    lda2.fit(Gtr2,Ytr2)

    Gte2 = csp2.transform(Xte)
    Gval2 = csp2.transform(Xval)
    #ldatest
    acc_sr = lda2.score(Gte2, Yte)
    # time
    stop = timeit.default_timer()
    time_sr = stop - start
    

    M = len(Yval)
    
    #%% # Sinkhorn Transport
    # Subset selection
    
    lda3 = LinearDiscriminantAnalysis()

    G_FOTDAs_, Y_FOTDAs_, regu_FOTDAs_=\
    SelectSubsetTraining_OTDAs(Gtr, Ytr, Gval, Yval, rango_e, lda3, metrica, outerkfold, innerkfold, M)

    #time
    start = timeit.default_timer()
    
    Gtr_daot=G_FOTDAs_
    Ytr_daot=Y_FOTDAs_ 
    ot_sinkhorn= ot.da.SinkhornTransport(metric=metrica, reg_e=regu_FOTDAs_)
    #learn the map
    ot_sinkhorn.fit(Xs=Gtr_daot, ys=Ytr_daot, Xt=Gval)
    #apply the mapping over source data
    transp_Xs_sinkhorn = ot_sinkhorn.transform(Xs=Gtr)

    # train a new classifier bases upon the transform source data
    lda3.fit(transp_Xs_sinkhorn,Ytr)
    # Compute acc
    yt_predict_1=lda3.predict(Gte)
    acc_fotdas=accuracy_score(Yte, yt_predict_1)
    # time
    stop = timeit.default_timer()
    time_fs = stop - start  
    #%% # Group-Lasso Transport
    # Subset selection
   
    lda3 = LinearDiscriminantAnalysis()

    G_FOTDAl1l2_, Y_FOTDAl1l2_, regu_FOTDAl1l2_=\
    SelectSubsetTraining_OTDAl1l2(Gtr, Ytr, Gval, Yval, rango_e, rango_cl, lda3, metrica, outerkfold, innerkfold, M)
    #time
    start = timeit.default_timer()
    
    Gtr_daot=G_FOTDAl1l2_
    Ytr_daot=Y_FOTDAl1l2_
    
    ot_l1l2 = ot.da.SinkhornL1l2Transport(metric=metrica ,reg_e=regu_FOTDAl1l2_[0], reg_cl=regu_FOTDAl1l2_[1])

    ot_l1l2.fit(Xs=Gtr_daot, ys=Ytr_daot, Xt=Gval)

    #transport taget samples onto source samples
    transp_Xs_l1l2=ot_l1l2.transform(Xs=Gtr)

    # train a new classifier bases upon the transform source data
    lda3.fit(transp_Xs_l1l2,Ytr)

    # Compute acc
    yt_predict_2=lda3.predict(Gte)
    acc_fotdal1l2=accuracy_score(Yte, yt_predict_2)
    # time
    stop = timeit.default_timer()
    time_fg = stop - start 
        
    #%% # Backward Sinkhorn Transport
    # Subset selection
    
    G_BOTDAs_, Y_BOTDAs_, regu_BOTDAs_=\
    SelectSubsetTraining_BOTDAs(Gtr, Ytr, Gval, Yval, rango_e, lda, metrica, outerkfold, innerkfold, M)
    # time
    start = timeit.default_timer()
    Gtr_botda=G_BOTDAs_
    Ytr_botda=Y_BOTDAs_
    
    bot_s = ot.da.SinkhornTransport(metric=metrica, reg_e=regu_BOTDAs_)
    
    
    bot_s.fit(Xs=Gval, ys=Yval, Xt=Gtr_botda)
    #transport testing samples
    transp_Xt_s_backward=bot_s.transform(Xs=Gte)
    # Compute accuracy without retraining    
    yt_predict_3=lda.predict(transp_Xt_s_backward)
    acc_botdas=accuracy_score(Yte, yt_predict_3)
    # time
    stop = timeit.default_timer()
    time_bs = stop - start
    
    #%% # Backward Group-Lasso Transport
    # Subset selection 
    G_BOTDAl1l2_, Y_BOTDAl1l2_, regu_BOTDAl1l2_=\
    SelectSubsetTraining_BOTDAl1l2(Gtr, Ytr, Gval, Yval, rango_e, rango_cl, lda, metrica, outerkfold, innerkfold, M)
    
    #time
    start = timeit.default_timer()
    Gtr_botda=G_BOTDAl1l2_
    Ytr_botda=Y_BOTDAl1l2_
    
    bot_l1l2 = ot.da.SinkhornL1l2Transport(metric=metrica, reg_e=regu_BOTDAl1l2_[0], reg_cl=regu_BOTDAl1l2_[1])
    
    bot_l1l2.fit(Xs=Gval, ys=Yval, Xt=Gtr_botda)
    #transport testing samples
    transp_Xt_l1l2_backward=bot_l1l2.transform(Xs=Gte)
    # Compute accuracy without retraining    
    yt_predict_4=lda.predict(transp_Xt_l1l2_backward)
    acc_botdal1l2=accuracy_score(Yte, yt_predict_4)
    # time
    stop = timeit.default_timer()
    time_bg = stop - start
    
    # Riemann
    # time
    start = timeit.default_timer()
    # cov matrix estimation
    cov_tr = Covariances().transform(Xtr)
    cov_val= Covariances().transform(Xval)
    cov_te = Covariances().transform(Xte)
        
    clf = MDM()
    source={'covs':cov_tr, 'labels': Ytr}
    target_org_train={'covs':cov_val, 'labels': Yval}
    target_org_test={'covs':cov_te, 'labels': Yte}
    # re-centered matrices
    source_rct, target_rct_train, target_rct_test = TL.RPA_recenter(source, target_org_train, target_org_test)   
    # rotate the re-centered-stretched matrices using information from classes
    source_rpa, target_rpa_train, target_rpa_test = TL.RPA_rotate(source_rct, target_rct_train, target_rct_test)
    # get data
    covs_source, y_source = source_rpa['covs'], source_rpa['labels']
    covs_target_train, y_target_train = target_rpa_train['covs'], target_rpa_train['labels']
    covs_target_test, y_target_test = target_rpa_test['covs'], target_rpa_test['labels']
    # append train and validation data
    covs_train = np.concatenate([covs_source, covs_target_train])
    y_train = np.concatenate([y_source, y_target_train])
    # train
    clf.fit(covs_train, y_train)
    # test
    covs_test = covs_target_test
    y_test = y_target_test
    y_pred = clf.predict(covs_test)
    #acc
    acc_rpa = accuracy_score(Yte, y_pred)
    # time
    stop = timeit.default_timer()
    time_rpa = stop - start
        
    # Euclidean
    # time
    start = timeit.default_timer()
    # Estimate single trial covariance
    cov_tr = Covariances().transform(Xtr)
    cov_val= Covariances().transform(Xval)
    Ctr = cov_tr.mean(0)
    Cval = cov_val.mean(0)
    # aligment
    Xtr_eu = np.asarray([np.dot(invsqrtm(Ctr), epoch) for epoch in Xtr])
    Xval_eu = np.asarray([np.dot(invsqrtm(Cval), epoch) for epoch in Xval])
    Xte_eu = np.asarray([np.dot(invsqrtm(Cval), epoch) for epoch in Xte])

    # append train and validation data
    x_train = np.concatenate([Xtr_eu, Xval_eu])
    y_train = np.concatenate([Ytr, Yval])

    # train new csp+lda
    csp2 = CSP(n_components=6, reg='empirical', log=True, norm_trace=False, cov_est='epoch')
    # learn csp filters
    Gtr2 = csp2.fit_transform(x_train,y_train)
    # learn lda
    lda2 = LinearDiscriminantAnalysis()

    lda2.fit(Gtr2,y_train)
    
    # test
    Gte2=csp2.transform(Xte_eu)  
    # acc
    acc_eu=lda2.score(Gte2, Yte)
    # time
    stop = timeit.default_timer()
    time_eu = stop - start
        
    
    # print results
    # accuracy
    acc = {}
    acc["sc"] = acc_sc
    acc["sr"] = acc_sr
    acc["rpa"] = acc_rpa
    acc["ea"] = acc_eu
    acc["fotda_s"] = acc_fotdas
    acc["fotda_l1l2"] = acc_fotdal1l2
    acc["botda_s"] = acc_botdas
    acc["botda_l1l2"] = acc_botdal1l2
    
    # computing time
    time = {}
    time["sr"] = round(time_sr,3)
    time["rpa"] = round(time_rpa,3)
    time["eu"] = round(time_eu,3)
    time["fotda_s"] = round(time_fs,3)
    time["fotda_l1l2"] = round(time_fg,3)
    time["botda_s"] = round(time_bs,3)
    time["botda_l1l2"] = round(time_bg,3)
    
    print('ACC')
    print(acc)
    print('CT')
    print(time)
    

Running testing RUN=0
ACC
{'sc': 0.55, 'sr': 0.55, 'rpa': 0.75, 'ea': 0.7, 'fotda_s': 0.75, 'fotda_l1l2': 0.65, 'botda_s': 0.9, 'botda_l1l2': 0.9}
CT
{'sr': 0.343, 'rpa': 1.892, 'eu': 0.407, 'fotda_s': 0.004, 'fotda_l1l2': 0.052, 'botda_s': 0.003, 'botda_l1l2': 0.03}
Running testing RUN=1
ACC
{'sc': 0.7, 'sr': 0.8, 'rpa': 0.8, 'ea': 0.85, 'fotda_s': 0.8, 'fotda_l1l2': 0.8, 'botda_s': 0.75, 'botda_l1l2': 0.85}
CT
{'sr': 0.324, 'rpa': 3.249, 'eu': 0.553, 'fotda_s': 0.004, 'fotda_l1l2': 0.091, 'botda_s': 0.004, 'botda_l1l2': 0.047}
Running testing RUN=2
ACC
{'sc': 0.65, 'sr': 0.75, 'rpa': 0.8, 'ea': 0.7, 'fotda_s': 0.75, 'fotda_l1l2': 0.8, 'botda_s': 0.8, 'botda_l1l2': 0.8}
CT
{'sr': 0.358, 'rpa': 3.038, 'eu': 0.596, 'fotda_s': 0.004, 'fotda_l1l2': 0.073, 'botda_s': 0.004, 'botda_l1l2': 0.056}
Running testing RUN=3
ACC
{'sc': 0.75, 'sr': 0.8, 'rpa': 0.8, 'ea': 0.8, 'fotda_s': 0.8, 'fotda_l1l2': 0.8, 'botda_s': 0.8, 'botda_l1l2': 0.8}
CT
{'sr': 0.362, 'rpa': 3.536, 'eu': 0.652, 'fotda_s': 

In [ ]:
#Plotting-->for the last testing run  

#distribution datasets
pl.figure(figsize=(8, 4))

pl.subplot(1, 3, 1)
pl.scatter(Gtr[Ytr==1, 0], Gtr[Ytr==1, 1], c='orange', marker='o', label='Class 1')
pl.scatter(Gtr[Ytr==2, 0], Gtr[Ytr==2, 1], c='indigo', marker='+', label='Class 2')

pl.xticks([])
pl.yticks([])
pl.legend(loc=1)
pl.title('Source (training) set')

pl.subplot(1, 3, 2)
pl.scatter(Gval[Yval==1, 0], Gval[Yval==1, 1], c='orange', marker='o', label='Class 1')
pl.scatter(Gval[Yval==2, 0], Gval[Yval==2, 1], c='indigo', marker='+', label='Class 2')

pl.xticks([])
pl.yticks([])
pl.legend(loc=1)
pl.title('Transportation set')
pl.tight_layout()

    
pl.subplot(1, 3, 3)
pl.scatter(Gte[Yte==1, 0], Gte[Yte==1, 1], c='orange', marker='o', label='Class 1')
pl.scatter(Gte[Yte==2, 0], Gte[Yte==2, 1], c='indigo', marker='+', label='Class 2')

pl.xticks([])
pl.yticks([])
pl.legend(loc=1)
pl.title('Target (testing) set' )
pl.tight_layout()



In [ ]:
# plotting original and transported samples for each method
pl.figure(figsize=(10, 11))
    
pl.subplot(2, 2, 1)
pl.scatter(Gte[Yte==1, 0], Gte[Yte==1, 1], c='orange', marker='o',
           alpha=0.5)
pl.scatter(Gte[Yte==2, 0], Gte[Yte==2, 1], c='indigo', marker='o',
           alpha=0.5)
pl.scatter(transp_Xs_sinkhorn[Ytr==1, 0], transp_Xs_sinkhorn[Ytr==1, 1], c='orange',
           marker='+', s=50 )
pl.scatter(transp_Xs_sinkhorn[Ytr==2, 0], transp_Xs_sinkhorn[Ytr==2, 1], c='indigo',
           marker='+', s=50 )
pl.title('Transported source samples\n FOTDA-S', size=14)
pl.xticks([])
pl.yticks([])

#l1l2
pl.subplot(2, 2, 2)
pl.scatter(Gte[Yte==1, 0], Gte[Yte==1, 1], c='orange', marker='o',
           alpha=0.5)
pl.scatter(Gte[Yte==2, 0], Gte[Yte==2, 1], c='indigo', marker='o',
           alpha=0.5)
pl.scatter(transp_Xs_l1l2[Ytr==1, 0], transp_Xs_l1l2[Ytr==1, 1], c='orange',
           marker='+', s=50 )
pl.scatter(transp_Xs_l1l2[Ytr==2, 0], transp_Xs_l1l2[Ytr==2, 1], c='indigo',
           marker='+', s=50 )
pl.title('Transported source samples\n FOTDA-GL', size=14)
pl.xticks([])
pl.yticks([])


#S-B
pl.subplot(2, 2, 3)
pl.scatter(Gtr[Ytr==1, 0], Gtr[Ytr==1, 1], c='orange', marker='o',alpha=0.5)
pl.scatter(Gtr[Ytr==2, 0], Gtr[Ytr==2, 1], c='indigo', marker='o',alpha=0.5)

pl.scatter(transp_Xt_s_backward[Yte==1, 0], transp_Xt_s_backward[Yte==1, 1], c='orange',
            marker='+', s=50)
pl.scatter(transp_Xt_s_backward[Yte==2, 0], transp_Xt_s_backward[Yte==2, 1], c='indigo',
            marker='+', s=0)
pl.title('Transported target samples\n BOTDA-S', size=14)
pl.xticks([])
pl.yticks([])

    
#l1l2-B
pl.subplot(2, 2, 4)
pl.scatter(Gtr[Ytr==1, 0], Gtr[Ytr==1, 1], c='orange', marker='o',alpha=0.5, 
           Label='Source samples class-1')
pl.scatter(Gtr[Ytr==2, 0], Gtr[Ytr==2, 1], c='indigo', marker='o',alpha=0.5,
           Label='Source samples class-2')

pl.scatter(transp_Xt_l1l2_backward[Yte==1, 0], transp_Xt_l1l2_backward[Yte==1, 1], c='orange',
            marker='+', s=50, Label='Transp. Target samples class-1')
pl.scatter(transp_Xt_l1l2_backward[Yte==2, 0], transp_Xt_l1l2_backward[Yte==2, 1], c='indigo',
            marker='+', s=50, Label='Transp. Target samples class-2')
pl.title('Transported target samples\n BOTDA-GL', size=14)
pl.legend(loc=8, ncol=2, mode="expand", bbox_to_anchor=(0, -0.3, 1, 0.65))
pl.xticks([])
pl.yticks([])
pl.tight_layout()